In [1]:
! pip install transformers
! pip install sentencepiece
! pip install datasets sacrebleu torch transformers
! pip install accelerate -U
! pip install datasets


You should consider upgrading via the 'C:\Users\klouc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\klouc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\klouc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\klouc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\klouc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

from sklearn.model_selection import train_test_split

import pandas as pd

from datasets import Dataset, load_metric

import regex as re

import torch

a = 0
for b in range(0,10,2):
    a += b + 1
print(a)

C:\Users\klouc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



25


In [2]:
# BLEU = "bleu"
FILENAME = "errors.csv"

MODEL_CHECKPOINT = "google/mt5-base"
MODEL_NAME = MODEL_CHECKPOINT.split("/")[-1]

CORRECTION = "correction"
SOURCE = "erroneous"
TARGET = "correct"

SOURCE_LANG = "erroneous-bg"
TARGET_LANG = "correct-bg"

ERRORNEOUS_BULGARIAN_TEXT = "erroneous_bg_text"
ERRORNEOUS_BULGARIAN = "erroneous_bulgarian"
CORRECT_BULGARIAN_TEXT = "correct_bg_text"
CORRECT_BULGARIAN = "correct_bulgarian"

EPOCH = "epoch"
INPUT_IDS = "input_ids"
GEN_LEN = "gen_len"

LABELS = "labels"
PREFIX = ""
SCORE = "score"
UNNAMED_COL = "Unnamed: 0"

MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 1
EPOCH_NUM = 10
LEARNING_RATE = 2e-4
WEIGHT_DECAY = 0.02
print(torch.cuda.is_available())

True


In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
model = model.cuda()

print(tokenizer.tokenize("Какво се случва тук?"))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\klouc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tok

['▁Как', 'во', '▁се', '▁случ', 'ва', '▁тук', '?']


In [5]:
df = pd.read_csv('errors.csv')

# Clean from ___
df['erroneous'] = df['erroneous'].apply(lambda x: re.sub(r'___', r'', x))
df['correct'] = df['correct'].apply(lambda x: re.sub(r'___', r'', x))

LIMIT = 50
correct_mask = df['correct'].str.len() <= LIMIT
error_mask = df['erroneous'].str.len() <= LIMIT
mask = []
d = 0
for c,e in zip(correct_mask, error_mask):
    mask.append(c and e)
    if c and e:
        d += 1
df = df[pd.Series(mask)]

In [6]:
X = df['erroneous']
y = df['correct']

# Split into train, val, test (72/18/10)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                               test_size=0.10,
                                               shuffle=True,
                                               random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  random_state=42)
X.head(10)

2                   Гена влиза в състава на хромозомите.
6                Проекта започва на 7 септември 2004 г_.
33                   Проекта стартира на 10 юли 2003 г_.
77             Периода на лечение обаче е продължителен.
78                  Релефа на щата е хълмист и равнинен.
79      Почерка е един от най-сложните навици на човека.
99           Престола е зает от братовчед им, Луи-Филип.
103          HTTP протокола използва обикновено порт 80.
122         Процеса се провежда във вакуум при 1200 ° C.
134    Въпроса остава без отговор, въпреки_че ефективния
Name: erroneous, dtype: object

In [7]:
def prep_data_for_model_fine_tuning(source_lang: list,
                                    target_lang: list) -> list:
    """Takes the input data lists and converts into error/correct list of
    dicts"""

    data_dict = dict()
    data_dict[CORRECTION] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict[SOURCE] = sr_text
        temp_dict[TARGET] = tr_text

        data_dict[CORRECTION].append(temp_dict)

    return data_dict

def generate_model_ready_dataset(dataset: list, source: str, target: str,
                                 model_checkpoint: str,
                                 tokenizer: AutoTokenizer):
    """Makes the data training ready for the model"""

    preped_data = []

    for row in dataset:
        inputs = PREFIX + row[source]
        targets = row[target]

        # Tokenize source
        model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)

        model_inputs[CORRECTION] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=MAX_INPUT_LENGTH,
                                 truncation=True, padding=True)
            model_inputs[LABELS] = labels[INPUT_IDS]

        preped_data.append(model_inputs)

    return preped_data

# Prepare training, validation, and test data
training_data = prep_data_for_model_fine_tuning(X_train.values, y_train.values)
validation_data = prep_data_for_model_fine_tuning(X_val.values, y_val.values)
test_data = prep_data_for_model_fine_tuning(X_test.values, y_test.values)

# Generate model-ready inputs for training, validation, and test
train_data = generate_model_ready_dataset(dataset=training_data[CORRECTION],
                                          tokenizer=tokenizer,
                                          source=SOURCE,
                                          target=TARGET,
                                          model_checkpoint=MODEL_CHECKPOINT)
validation_data = generate_model_ready_dataset(dataset=validation_data[CORRECTION],
                                               tokenizer=tokenizer,
                                               source=SOURCE,
                                               target=TARGET,
                                               model_checkpoint=MODEL_CHECKPOINT)
test_data = generate_model_ready_dataset(dataset=test_data[CORRECTION],
                                         tokenizer=tokenizer,
                                         source=SOURCE,
                                         target=TARGET,
                                         model_checkpoint=MODEL_CHECKPOINT)

# Convert to DataFrame
train_df = pd.DataFrame.from_records(train_data)
validation_df = pd.DataFrame.from_records(validation_data)
test_df = pd.DataFrame.from_records(test_data)

# Convert DataFrames to Dataset objects
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

C:\Users\klouc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
# def compute_metrics(eval_preds: tuple) -> dict:
#     """computes bleu score and other performance metrics """

#     metric = load_metric("sacrebleu")
#     tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

#     preds, labels = eval_preds

#     if isinstance(preds, tuple):
#         preds = preds[0]

#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {BLEU: result[SCORE]}

#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

#     result[GEN_LEN] = np.mean(prediction_lens)
#     result = {k: round(v, 4) for k, v in result.items()}

#     return result

In [9]:
from transformers.trainer_callback import TrainerCallback

torch.cuda.empty_cache()

class LoggingCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        print(f"Epoch {state.epoch}: Training Loss {state.loss_meter.global_avg}")


model_args = Seq2SeqTrainingArguments(
    f"{MODEL_NAME}-finetuned-{SOURCE_LANG}-to-{TARGET_LANG}",
    evaluation_strategy=EPOCH,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=3,
    num_train_epochs=EPOCH_NUM,
    predict_with_generate=True
)

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   2224 MiB |   2224 MiB |   2224 MiB |      0 B   |\n|       from large pool |   2224 MiB |   2224 MiB |   2224 MiB |      0 B   |\n|       from small pool |      0 MiB |      0 MiB |      0 MiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |   2224 MiB |   2224 MiB |   2224 MiB |      0 B   |\n|       from large pool |   2224 MiB |   2224 MiB |

In [10]:
# Initialize the Seq2SeqTrainer for fine-tuning
trainer = Seq2SeqTrainer(
    model,
    model_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
        callbacks=[LoggingCallback()]
    # compute_metrics=compute_metrics
)

# Commence the model training
trainer.train()

# Save the fine-tuned model
trainer.save_model("FineTunedTransformer")

print(test_dataset)
print(trainer.predict(test_dataset))

  0%|          | 0/790 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 4.34 GiB already allocated; 0 bytes free; 4.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
DIR = './slovcho-grammar'
ft_model_tokenizer = AutoTokenizer.from_pretrained(f"{DIR}")
ft_model = AutoModelForSeq2SeqLM.from_pretrained(f"{DIR}")

In [9]:
ft_prediction = []

input = [
    'Договора е напълно ясен'
]

for sentence in input:
    encoded_text = ft_model_tokenizer(sentence, return_tensors='pt', max_length=MAX_INPUT_LENGTH,
                               padding='max_length')
    translated = ft_model.generate(**encoded_text, max_length=MAX_INPUT_LENGTH)
    print(translated)
    ft_prediction.append([ft_model_tokenizer.decode(t, skip_special_tokens=True) for t in translated][0])

ft_prediction

tensor([[     0,   4223,  14878,   4352,    259,    324,  70864, 114175,    259,
         177180,      1]])


['Договорът е напълно ясен']